In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
import json


In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [ ]:

class ProjectManagerAgent:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-4", temperature=0.7)
        self.memory = ConversationBufferMemory()
    
    def parse_user_input(self, user_idea: str) -> dict:
       
        prompt = f"""
        You are an AI Project Manager. Your responsibility is to take a given business idea (provided by the user) and break it down into 3 
        detailed tasks for the following agents:

        • **Engineering Agent**: Handles technical design, system architecture, and component designs. The agent does not have to give any code,
        but should provide a detailed technical overview of the system.
        • **Marketing Agent**: Focuses on market research, branding, and promotional strategy.
        • **Legal Agent**: Deals with compliance, risk assessment, and legal documentation.

        Your output must be a valid JSON object with exactly 4 key-value pairs. Each key is the name of an agent (use lowercase strings 
        exactly as "engineering", "marketing", "legal"), and each value is a detailed prompt instructing that 
        agent on its specific task.

        The JSON format should look like this:
        {{
            "engineering": "Detailed task prompt for the Engineering Agent.",
            "marketing": "Detailed task prompt for the Marketing Agent.",
            "legal": "Detailed task prompt for the Legal Agent.",
           
        }}
        Each task prompt should provide the original user idea and clearly specify objectives, deliverables, and any constraints derived from the user’s business idea. Return only the JSON object with no additional text.

        Business Idea: {user_idea}
        """

        response = self.llm.invoke(prompt).content
        print(response)
        structured_tasks = json.loads(response)
        return structured_tasks
    

In [8]:
pm_agent = ProjectManagerAgent()
user_idea = "An AI-powered chatbot that helps small businesses create automated marketing content."
structured_tasks = pm_agent.parse_user_input(user_idea)

for agent, task in structured_tasks.items():
    print(f"\n{agent.capitalize()} Task:\n{task}")

{
    "engineering": "The project is to develop an AI-powered chatbot that aids small businesses in automatically creating marketing content. Your task is to design an intelligent system that can comprehend user requirements and generate suitable marketing content. The bot should be able to take input in the form of business details, target audience, and marketing goals, and output relevant content. The system should be user-friendly, adaptable to multiple platforms, and have a smart content generation algorithm at its core. Deliverables include a detailed technical design document, system architecture, and component designs. Please ensure the system is scalable and performance optimized.",

    "marketing": "The product is an AI-powered chatbot for small businesses to generate marketing content. Conduct thorough market research to understand our potential customers, their needs, and the competitive landscape. Develop a branding strategy that highlights our bot's user-friendly interfac

In [27]:
dummy_response = {
    "Engineering": {
        "System Architecture": "The chatbot system follows a modular architecture with three core components: a Natural Language Processing (NLP) module powered by GPT-4 for understanding and generating marketing content, a Content Recommendation Engine leveraging machine learning models to suggest relevant marketing materials, and a User Interface (UI) layer for seamless interaction via web and mobile apps. Data storage will utilize a cloud-based database with secure encryption to handle user inputs and generated content.",
        "Components List": [
            "GPT-4 NLP Model for language understanding and content generation",
            "Machine Learning Recommendation Engine for marketing strategies",
            "Cloud-based database for storing user interactions and generated content",
            "Frontend UI (React.js for web, Flutter for mobile) for user engagement",
            "Backend API (FastAPI/Python) for processing requests and integrating AI components"
        ],
        "Technical Specification": {
            "Programming Languages": ["Python", "JavaScript"],
            "Frameworks": ["FastAPI for backend", "React.js for frontend", "Flutter for mobile"],
            "Database": "PostgreSQL with cloud hosting",
            "Security Measures": ["OAuth 2.0 authentication", "End-to-end encryption", "GDPR-compliant data handling"]
        }
    },
    "Marketing": {
        "Market Research": {
            "Current Trends": [
                "Increasing adoption of AI in digital marketing",
                "Small businesses looking for cost-effective automation solutions",
                "Preference for no-code or low-code marketing tools"
            ],
            "Key Competitors": [
                "HubSpot AI-powered marketing tools",
                "Copy.ai for AI-generated content",
                "ChatGPT-based customer engagement tools"
            ],
            "Customer Pain Points": [
                "High costs of hiring marketing agencies",
                "Time-consuming manual content creation",
                "Lack of personalized marketing strategies"
            ]
        },
        "Branding Strategy": {
            "Unique Selling Proposition": "An AI chatbot that provides small businesses with personalized, automated marketing content effortlessly.",
            "Brand Persona": "Helpful, smart, and easy-to-use marketing assistant for small business owners.",
            "Key Messaging": "Empower your business with AI-driven marketing content, tailored for your audience."
        },
        "Promotional Plan": {
            "Digital Channels": ["Social media (LinkedIn, Facebook, Instagram)", "Email marketing campaigns", "Content marketing through blogs and case studies"],
            "Initial Campaign": "A free trial launch campaign offering businesses AI-generated marketing content for 30 days.",
            "Advertising Budget": "Targeted paid ads with a budget of $10,000 for the first quarter."
        }
    },
    "Legal": {
        "Risk Assessment": {
            "Potential Risks": [
                "User data privacy concerns under GDPR and CCPA regulations",
                "AI-generated content misuse (e.g., plagiarism, misleading ads)",
                "Liability issues related to marketing claims made using AI content"
            ],
            "Mitigation Strategies": [
                "Implement clear opt-in and opt-out user consent for data collection",
                "Develop an AI content monitoring system to flag potential policy violations",
                "Include disclaimers in generated content stating that AI assists in content creation"
            ]
        },
        "Legal Documents": {
            "Terms Of Use": "A document outlining permissible uses of the chatbot, limitations of liability, and user responsibilities.",
            "Privacy Policy": "A comprehensive privacy policy detailing data collection, storage, and user rights under GDPR and CCPA.",
            "Compliance Measures": [
                "Data encryption for user information",
                "Regular AI ethics audits",
                "Adherence to fair advertising laws"
            ]
        }
    }
}


In [ ]:
import json
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx2pdf import convert


json_data = dummy_response
# Create a new Word document
doc = Document()

doc.add_heading('Project Documentation', level=1)


for main_section, subsections in json_data.items():
    doc.add_heading(main_section, level=1)  
    
    for sub_heading, content in subsections.items():
        doc.add_heading(sub_heading, level=2)  

        # Handle content based on type
        if isinstance(content, list):  
            for item in content:
                doc.add_paragraph(f"• {item}", style="List Bullet")
        elif isinstance(content, dict):  
            for key, value in content.items():
                doc.add_heading(key, level=3)
                if isinstance(value, list):
                    for sub_item in value:
                        doc.add_paragraph(f"• {sub_item}", style="List Bullet")
                else:
                    doc.add_paragraph(value)
        else:  # If the value is a string, just add it as a paragraph
            doc.add_paragraph(content)

# Save as a Word document
docx_filename = r"docs/Project_Documentation.docx"
doc.save(docx_filename)

# Convert Word document to PDF
pdf_filename = "Project_Documentation.pdf"
convert(docx_filename)

print(f"Document successfully saved as {pdf_filename}.")


  0%|          | 0/1 [00:00<?, ?it/s]

Document successfully saved as Project_Documentation.pdf.
